# PROYECTO PROCESAMIENTO LENGUAJE NATURAL MINI-LLAMA
Sebastián Insuasti

En este notebook se ejecutará los códigos guía sugerios en el proyecto para revisar si la implementación del Mini Llama fue satisfactoria. Así también, se realiza gráficas de los resultados más destacables

## Reference Commands and Expected Results

In [1]:
from run_llama import get_args, generate_sentence, test_with_prompting, train, test, train_lora, seed_everything
import warnings
warnings.filterwarnings("ignore")
import time

### Definición de Clase Args 

Se modifica la implementación original para que se pueda implementar de forma más amigable en "notebook" y no únicamente con los comandos predeterminados en el terminal.

In [2]:
class Args:
    def __init__(self, option="generate", epochs=5, lr=2e-5, seed=1337,
                 train="data/cfimdb-train.txt", dev="data/cfimdb-dev.txt",
                 test="data/cfimdb-test.txt", label_names="data/cfimdb-label-mapping.json",
                 pretrained_model_path="stories42M.pt", max_sentence_len=None,
                 use_gpu=True, generated_sentence_low_temp_out="generated-sentence-temp-0.txt",
                 generated_sentence_high_temp_out="generated-sentence-temp-1.txt",
                 dev_out="cfimdb-dev-prompting-output.txt", test_out="cfimdb-test-prompting-output.txt",
                 lora_rank=4, lora_alpha=1.0, batch_size=8, hidden_dropout_prob=0.3):
        
        self.option = option
        self.epochs = epochs
        self.lr = lr
        self.seed = seed
        self.train = train
        self.dev = dev
        self.test = test
        self.label_names = label_names
        self.pretrained_model_path = pretrained_model_path
        self.max_sentence_len = max_sentence_len
        self.use_gpu = use_gpu
        self.generated_sentence_low_temp_out = generated_sentence_low_temp_out
        self.generated_sentence_high_temp_out = generated_sentence_high_temp_out
        self.dev_out = dev_out
        self.test_out = test_out
        self.lora_rank = lora_rank
        self.lora_alpha = lora_alpha
        self.batch_size = batch_size
        self.hidden_dropout_prob = hidden_dropout_prob



### Time

Se declara una función de Tiempo para determinar el tiempo de ejecución de las funciones implementadas para hacer text generation, zero-shot, finetuning and lora finetuning. Se utiliza esta función como un decorador para envolver todas las funciones que se desea implementar.

In [3]:
def timed(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"Tiempo: {func.__name__} ejecutada en {end - start:.4f} segundos")
        return result
    return wrapper

timed_generate_sentence = timed(generate_sentence)
timed_test_with_prompting = timed(test_with_prompting)
timed_train = timed(train)
timed_train_lora = timed(train_lora)
timed_test = timed(test)


## Text Generation

**Comando Original**:
python run_llama.py --option generate

In [4]:
args = Args()
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)

prefix = "Victor Frankenstein, a young scientist obsessed with uncovering the secrets of life, succeeds in animating a creature assembled from"

timed_generate_sentence(args, prefix, args.generated_sentence_low_temp_out, max_new_tokens=75, temperature=0.0)

load model from stories42M.pt
Temperature is 0.0
Victor Frankenstein, a young scientist obsessed with uncovering the secrets of life, succeeds in animating a creature assembled from the unknown. Everyone in the village was curious to know what the creature was.
One day, a young girl called Lily asked, "What is unlocking?"
Lily smiled and said, "I am unlocking the mystery of the unknown."
Lily was excited to find out what the mystery was. She was determined to unlock the
---------------
Wrote generated sentence to generated-sentence-temp-0.txt.
Tiempo: generate_sentence ejecutada en 3.2544 segundos


In [30]:
timed_generate_sentence(args, prefix, args.generated_sentence_high_temp_out, max_new_tokens=75, temperature=1.0)

load model from stories42M.pt
Temperature is 1.0
Victor Frankenstein, a young scientist obsessed with uncovering the secrets of life, succeeds in animating a creature assembled from the unknown. One day, a three year old boy called Timmy and his parents were visiting his grandparents. Timmy was so excited to show them what he had been dreaming of.
"What did he have dream of," his dad asked. 
Timmy's dad smiled. He was happy to tell Timmy the story of
---------------
Wrote generated sentence to generated-sentence-temp-1.txt.
Tiempo: generate_sentence ejecutada en 1.9764 segundos


**Generación Mejorada al Implementar top_k y top_p al sampler**

In [5]:
args = Args()
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)

prefix = "Victor Frankenstein, a young scientist obsessed with uncovering the secrets of life, succeeds in animating a creature assembled from"

timed_generate_sentence(args, prefix, args.generated_sentence_low_temp_out, max_new_tokens=75, temperature=0.7, top_k=50, top_p=0.5)

load model from stories42M.pt
Temperature is 0.7
Victor Frankenstein, a young scientist obsessed with uncovering the secrets of life, succeeds in animating a creature assembled from the unknown. One day, a young girl called Lucy was walking in the woods. She was looking for something. Suddenly, Lucy saw the creature and ran away. Lucy was scared and ran home.
The next day, Lucy was brave and faced the unknown creature. She was scared, but she kept running. Suddenly, the creature was caught
---------------
Wrote generated sentence to generated-sentence-temp-0.txt.
Tiempo: generate_sentence ejecutada en 1.9861 segundos


## Zero-Shot Prompting

### SST Dataset
**Comando Original**:
python run_llama.py --option prompt --batch_size 10 --train data/sst-train.txt --dev data/sst-dev.txt --test data/sst-test.txt --label-names data/sst-label-mapping.json --dev_out sst-dev-prompting-output.txt --test_out sst-test-prompting-output.txt [--use_gpu]

In [31]:
args = Args(batch_size=10, train="data/sst-train.txt", dev="data/sst-dev.txt", test="data/sst-test.txt", label_names="data/sst-label-mapping.json", dev_out="sst-dev-prompting-output.txt", test_out="sst-test-prompting-output.txt")
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)

timed_test_with_prompting(args)

load 8544 data from data/sst-train.txt
load 1101 data from data/sst-dev.txt
load 2210 data from data/sst-test.txt


eval: 100%|██████████| 221/221 [00:04<00:00, 53.00it/s]


dev acc :: 0.237
test acc :: 0.250
Tiempo: test_with_prompting ejecutada en 7.9520 segundos


### CFIMDB Dataset
**Comando Original**: python run_llama.py --option prompt --batch_size 10 --train data/cfimdb-train.txt --dev data/cfimdb-dev.txt --test data/cfimdb-test.txt --label-names data/cfimdb-label-mapping.json --dev_out cfimdb-dev-prompting-output.txt --test_out cfimdb-test-prompting-output.txt [--use_gpu]

In [32]:
args = Args(batch_size=10, train="data/cfimdb-train.txt", dev="data/cfimdb-dev.txt", test="data/cfimdb-test.txt", 
            label_names="data/cfimdb-label-mapping.json", dev_out="cfimdb-dev-prompting-output.txt", 
            test_out="cfimdb-test-prompting-output.txt")
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)
timed_test_with_prompting(args)

load 1707 data from data/cfimdb-train.txt
load 245 data from data/cfimdb-dev.txt
load 488 data from data/cfimdb-test.txt


eval: 100%|██████████| 49/49 [00:04<00:00, 10.25it/s]

dev acc :: 0.490
test acc :: 0.109
Tiempo: test_with_prompting ejecutada en 9.3763 segundos


## Classification Fine-Tuning

### SST Dataset


**Comando Original**: python run_llama.py --option finetune --epochs 5 --lr 2e-5 --batch_size 80 --train data/sst-train.txt --dev data/sst-dev.txt --test data/sst-test.txt --label-names data/sst-label-mapping.json --dev_out sst-dev-finetuning-output.txt --test_out sst-test-finetuning-output.txt

In [33]:
args = Args(epochs=5, batch_size=80, lr=2e-5, train="data/sst-train.txt", dev="data/sst-dev.txt", test="data/sst-test.txt", 
            label_names="data/sst-label-mapping.json", dev_out="sst-dev-finetuning-output.txt", 
            test_out="sst-test-finetuning-output.txt")
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)
print("---TRAIN---")
timed_train(args)

print("---TEST---")
timed_test(args)

---TRAIN---
load 8544 data from data/sst-train.txt
load 1101 data from data/sst-dev.txt


 20%|██        | 1/5 [00:45<03:00, 45.03s/it]

save the model to generate-5-2e-05.pt
epoch 0: train loss :: 1.905, train acc :: 0.262, dev acc :: 0.265


 40%|████      | 2/5 [01:30<02:15, 45.31s/it]

save the model to generate-5-2e-05.pt
epoch 1: train loss :: 1.641, train acc :: 0.304, dev acc :: 0.278


...

100%|██████████| 5/5 [03:49<00:00, 45.92s/it]

save the model to generate-5-2e-05.pt
epoch 4: train loss :: 0.975, train acc :: 0.761, dev acc :: 0.400
Tiempo: train ejecutada en 231.0661 segundos
---TEST---


load model from generate-5-2e-05.pt
load 1101 data from data/sst-dev.txt
load 2210 data from data/sst-test.txt


eval: 100%|██████████| 28/28 [00:02<00:00, 10.09it/s]


dev acc :: 0.400
test acc :: 0.408
Tiempo: test ejecutada en 5.9013 segundos


### CFIMDB DataSet

**Comando Original**:
python run_llama.py --option finetune --epochs 5 --lr 2e-5 --batch_size 10 --train data/cfimdb-train.txt --dev data/cfimdb-dev.txt --test data/cfimdb-test.txt --label-names data/cfimdb-label-mapping.json --dev_out cfimdb-dev-finetuning-output.txt --test_out cfimdb-test-finetuning-output.txt [--use_gpu]

In [34]:
args = Args(batch_size=10, epochs=5, lr=2e-5, train="data/cfimdb-train.txt", dev="data/cfimdb-dev.txt", test="data/cfimdb-test.txt", 
            label_names="data/cfimdb-label-mapping.json", dev_out="cfimdb-dev-finetuning-output.txt", 
            test_out="cfimdb-test-finetuning-output.txt")
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)

print("---TRAIN---")
timed_train(args)

print("---TEST---")
timed_test(args)

---TRAIN---
load 1707 data from data/cfimdb-train.txt
load 245 data from data/cfimdb-dev.txt


 20%|██        | 1/5 [02:30<10:01, 150.27s/it]

save the model to generate-5-2e-05.pt
epoch 0: train loss :: 0.992, train acc :: 0.501, dev acc :: 0.502


 40%|████      | 2/5 [04:59<07:28, 149.55s/it]

save the model to generate-5-2e-05.pt
epoch 1: train loss :: 0.799, train acc :: 0.624, dev acc :: 0.629


...

100%|██████████| 5/5 [12:34<00:00, 150.84s/it]

save the model to generate-5-2e-05.pt
epoch 4: train loss :: 0.402, train acc :: 0.896, dev acc :: 0.833
Tiempo: train ejecutada en 755.9370 segundos
---TEST---


load model from generate-5-2e-05.pt
load 245 data from data/cfimdb-dev.txt
load 488 data from data/cfimdb-test.txt


eval: 100%|██████████| 49/49 [00:08<00:00,  5.83it/s]

dev acc :: 0.833
test acc :: 0.590
Tiempo: test ejecutada en 15.5409 segundos


## LoRA Fine-Tuning

### SST DataSet
**Comando Original**:
python run_llama.py --option lora --epochs 5 --lr 2e-5 --batch_size 80 --train data/sst-train.txt --dev data/sst-dev.txt --test data/sst-test.txt --label-names data/sst-label-mapping.json --dev_out sst-dev-lora-output.txt --test_out sst-test-lora-output.txt --lora_rank 4 --lora_alpha 1.0 [--use_gpu]

In [35]:
args = Args(epochs=5, batch_size=80, lr=2e-5, train="data/sst-train.txt", dev="data/sst-dev.txt", test="data/sst-test.txt", 
            label_names="data/sst-label-mapping.json", dev_out="sst-dev-lora-output.txt", 
            test_out="sst-test-lora-output.txt", lora_rank=4, lora_alpha=1.0)
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)

print("---TRAIN---")
timed_train_lora(args)

print("---TEST---")
timed_test(args)

---TRAIN---
load 8544 data from data/sst-train.txt
load 1101 data from data/sst-dev.txt
Applying LoRA with rank=4, alpha=1.0
Applied LoRA to layers.0.attention.compute_query
Applied LoRA to layers.0.attention.compute_key
Applied LoRA to layers.0.attention.compute_value
Applied LoRA to layers.0.attention.compute_output
Applied LoRA to layers.1.attention.compute_query
Applied LoRA to layers.1.attention.compute_key
Applied LoRA to layers.1.attention.compute_value
Applied LoRA to layers.1.attention.compute_output
Applied LoRA to layers.2.attention.compute_query
Applied LoRA to layers.2.attention.compute_key
Applied LoRA to layers.2.attention.compute_value
Applied LoRA to layers.2.attention.compute_output
Applied LoRA to layers.3.attention.compute_query
Applied LoRA to layers.3.attention.compute_key
Applied LoRA to layers.3.attention.compute_value
Applied LoRA to layers.3.attention.compute_output
Applied LoRA to layers.4.attention.compute_query
Applied LoRA to layers.4.attention.compute_key

eval: 100%|██████████| 14/14 [00:01<00:00,  8.82it/s]


Merging LoRA weights back into original model...


 20%|██        | 1/5 [00:47<03:10, 47.71s/it]

save the model to generate-5-2e-05.pt
Saved merged model to generate-5-2e-05.pt
LoRA epoch 0: train loss :: 3.830, train acc :: 0.235, dev acc :: 0.233


...

100%|██████████| 5/5 [03:59<00:00, 47.93s/it]

save the model to generate-5-2e-05.pt
Saved merged model to generate-5-2e-05.pt
LoRA epoch 4: train loss :: 2.312, train acc :: 0.264, dev acc :: 0.268
Tiempo: train_lora ejecutada en 241.3057 segundos
---TEST---


load model from generate-5-2e-05.pt
load 1101 data from data/sst-dev.txt
load 2210 data from data/sst-test.txt


eval: 100%|██████████| 28/28 [00:02<00:00,  9.91it/s]

dev acc :: 0.268
test acc :: 0.256
Tiempo: test ejecutada en 5.7941 segundos


### CFIMDB DataSet

**Comando Original**: python run_llama.py --option lora --epochs 5 --lr 2e-5 --batch_size 10 --train data/cfimdb-train.txt --dev data/cfimdb-dev.txt --test data/cfimdb-test.txt --label-names data/cfimdb-label-mapping.json --dev_out cfimdb-dev-lora-output.txt --test_out cfimdb-test-lora-output.txt --lora_rank 4 --lora_alpha 1.0 [--use_gpu]

In [36]:
args = Args(batch_size=10, epochs=5, lr=2e-5, train="data/cfimdb-train.txt", dev="data/cfimdb-dev.txt", test="data/cfimdb-test.txt", 
            label_names="data/cfimdb-label-mapping.json", dev_out="cfimdb-dev-lora-output.txt", 
            test_out="cfimdb-test-lora-output.txt", lora_rank=4, lora_alpha=1.0)
args.filepath = f'{args.option}-{args.epochs}-{args.lr}.pt'
seed_everything(args.seed)
print("---TRAIN---")
timed_train_lora(args)
print("---TEST---")
timed_test(args)

---TRAIN---
load 1707 data from data/cfimdb-train.txt
load 245 data from data/cfimdb-dev.txt
Applying LoRA with rank=4, alpha=1.0
Applied LoRA to layers.0.attention.compute_query
Applied LoRA to layers.0.attention.compute_key
Applied LoRA to layers.0.attention.compute_value
Applied LoRA to layers.0.attention.compute_output
Applied LoRA to layers.1.attention.compute_query
Applied LoRA to layers.1.attention.compute_key
Applied LoRA to layers.1.attention.compute_value
Applied LoRA to layers.1.attention.compute_output
Applied LoRA to layers.2.attention.compute_query
Applied LoRA to layers.2.attention.compute_key
Applied LoRA to layers.2.attention.compute_value
Applied LoRA to layers.2.attention.compute_output
Applied LoRA to layers.3.attention.compute_query
Applied LoRA to layers.3.attention.compute_key
Applied LoRA to layers.3.attention.compute_value
Applied LoRA to layers.3.attention.compute_output
Applied LoRA to layers.4.attention.compute_query
Applied LoRA to layers.4.attention.comput

eval: 100%|██████████| 25/25 [00:06<00:00,  3.91it/s]


Merging LoRA weights back into original model...


 20%|██        | 1/5 [02:28<09:53, 148.48s/it]

save the model to generate-5-2e-05.pt
Saved merged model to generate-5-2e-05.pt
LoRA epoch 0: train loss :: 1.195, train acc :: 0.502, dev acc :: 0.518


...

LoRA epoch 3: train loss :: 1.104, train acc :: 0.523, dev acc :: 0.478


100%|██████████| 5/5 [11:56<00:00, 143.29s/it]


LoRA epoch 4: train loss :: 1.059, train acc :: 0.511, dev acc :: 0.490
Tiempo: train_lora ejecutada en 718.2622 segundos
---TEST---
load model from generate-5-2e-05.pt
load 245 data from data/cfimdb-dev.txt
load 488 data from data/cfimdb-test.txt


eval: 100%|██████████| 49/49 [00:08<00:00,  5.71it/s]

dev acc :: 0.518
test acc :: 0.424
Tiempo: test ejecutada en 15.3864 segundos
